# Assignment 1

**Due to**: 11/01/2022 (dd/mm/yyyy)

If you deliver it by 11/12/2021 your assignment will be graded by 11/01/2022.


**Credits**: Andrea Galassi, Federico Ruggeri, Paolo Torroni

**Summary**: Part-of Speech (POS) tagging as Sequence Labelling using Recurrent Neural Architectures

# Execution



In [14]:
import nltk
import numpy as np

nltk.corpus.download('treebank')

## 1. Corpus Pre-processing
### 1.1 Vocabulary

In [72]:
fileids = nltk.corpus.treebank.fileids()


# Train
train_tagged_words = nltk.corpus.treebank.tagged_words(fileids[:101])
vocabulary_1 = list(set(train_tagged_words))

print('Train set:')
print('\t',len(train_tagged_words))
print('\t',len(vocabulary_1))

# train_freqs = nltk.FreqDist(tag for (word, tag) in train_tagged_words)
# print(train_freqs.most_common())
# print(train_freqs.tabulate())

# Val
val_tagged_words = nltk.corpus.treebank.tagged_words(fileids[101:151])
vocabulary_2 = list(set(val_tagged_words))

print('Validation set:')
print('\t',len(val_tagged_words))
print('\t',len(vocabulary_2))

# val_freqs = nltk.FreqDist(tag for (word, tag) in val_tagged_words)
# print(val_freqs.most_common())

# Test
test_tagged_words = nltk.corpus.treebank.tagged_words(fileids[151:])
vocabulary_3 = list(set(test_tagged_words))

print('Test set:')
print('\t',len(test_tagged_words))
print('\t',len(vocabulary_3))

# test_freqs = nltk.FreqDist(tag for (word, tag) in test_tagged_words)
# print(test_freqs.most_common())

['#', '$', "''", ',', '-LRB-', '-NONE-', '-RRB-', '.', ':', 'CC', 'CD', 'DT', 'EX', 'FW', 'IN', 'JJ', 'JJR', 'JJS', 'LS', 'MD', 'NN', 'NNP', 'NNPS', 'NNS', 'PDT', 'POS', 'PRP', 'PRP$', 'RB', 'RBR', 'RBS', 'RP', 'SYM', 'TO', 'UH', 'VB', 'VBD', 'VBG', 'VBN', 'VBP', 'VBZ', 'WDT', 'WP', 'WP$', 'WRB', '``']
Train set:
	 51625
	 9374
Validation set:
	 32629
	 6473
Test set:
	 16422
	 3852


In [81]:
tags = sorted(list(set([x[1] for x in vocabulary_1])))
tags = zip(tags, list(range(len(tags))))
tags_dict = dict(tags)


vocabulary_1_idx = [(x[0], tags_dict[x[1]]) for x in vocabulary_1]
vocabulary_2_idx = [(x[0], tags_dict[x[1]]) for x in vocabulary_2]
vocabulary_3_idx = [(x[0], tags_dict[x[1]]) for x in vocabulary_3]

# print(vocabulary_1[:5])
# print(vocabulary_1_idx[:5])

[('Bretz', 'NNP'), ('*T*-34', '-NONE-'), ('attempting', 'VBG'), ('caused', 'VBN'), ('*T*-201', '-NONE-')]
[('Bretz', 21), ('*T*-34', 5), ('attempting', 37), ('caused', 38), ('*T*-201', 5)]


###1.2 Train test split


( (S 
    (NP-SBJ 
      (NP (NNP Pierr
